In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, find_peaks_cwt
from SecondaryValue import SecondaryValue

In [166]:
komm_1 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/kommerzieller_HeNe/4.1spektr.txt',
            skiprows=4)
komm_p1 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/kommerzieller_HeNe/pol1.txt',
            skiprows=1) * 1e4
komm_p2 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/kommerzieller_HeNe/pol2.txt',
            skiprows=4)


In [167]:
komm_1_narrowed = komm_1[20:250]

In [168]:
peaks, peak_info = find_peaks(komm_1_narrowed[:,1], height=80, width=1, rel_height=.5)
all_peaks, _ = find_peaks(komm_1[:,1], height=60, distance=10)

In [169]:
%matplotlib qt5
plt.plot(*komm_1_narrowed.T, label='Spektrum')
plt.plot(komm_1_narrowed[:,0][peaks[1::2]], komm_1_narrowed[:,1][peaks[1::2]], "x",
         color='red', label='Peaks')
plt.xlabel('Zeit [u]')
plt.ylabel('Amplitude [mV]')
plt.legend()
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [83]:
dt = .5
fsr = 1.99862e9, 0.067e9

In [84]:
peak_t = komm_1_narrowed[:,0][peaks]

In [99]:
unit = SecondaryValue('f/(x2-x1)')(f=fsr, x2=(peak_t[3], 1), x1=(peak_t[1], 1))


In [100]:
unit

(17229482.75862069, 614595.9341925731)

In [175]:
def calibrate_unit(ydata, thresh):
    xdata = np.arange(0, len(ydata))
    peaks, peak_info = find_peaks(ydata, height=thresh, width=1, rel_height=.5)

    plt.clf()
    plt.plot(xdata, ydata, label='Spektrum')
    plt.plot(xdata[peaks], ydata[peaks], "x",
             color='red', label='Peaks')
    plt.xlabel('Zeit [u]')
    plt.ylabel('Amplitude [mV]')
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.show()

    return SecondaryValue('f/(x2-x1)')(f=fsr, x2=(peaks[1], 1), x1=(peaks[0], 1))




In [177]:
calibrate_unit(komm_1_narrowed[:,1], 120)

(17229482.75862069, 614595.9341925731)

In [101]:
widths = peak_info['widths']
width = widths.mean()
d_width = widths.std()/2

In [102]:
width, d_width

(4.7190589321125325, 0.30525965381031983)

In [103]:
w = SecondaryValue('f*u')(f=(width, d_width), u=unit)

In [104]:
w

(81306944.50774784, 6006147.334780343)

In [105]:
fin = SecondaryValue('fsr/fw')(fsr=fsr, fw=w)

In [106]:
fin

(24.581172150793947, 1.9940442123908217)

In [108]:
plt.clf()
p1_fixed = komm_p1-komm_p1.min()
plt.plot(np.arange(0, len(komm_p1)), p1_fixed/p1_fixed.max())
plt.plot(np.arange(0, len(komm_p2)), komm_p2/komm_p2.max())
plt.show()

In [118]:
%matplotlib qt5
plt.plot(*komm_1.T, label='Spektrum')
plt.plot(komm_1[:,0][all_peaks], komm_1[:,1][all_peaks], "x",
         color='red', label='Peaks')
plt.xlabel('Zeit [u]')
plt.ylabel('Amplitude [mV]')
plt.legend()
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [110]:
dists = all_peaks[1::2] - all_peaks[0::2]
dist = dists.mean(), dists.std()/np.sqrt(len(dists))
real_dist = SecondaryValue('u*d')(u=unit, d=dist)

In [111]:
real_dist

(647828551.724138, 44686897.20641578)

In [112]:
L = SecondaryValue('c/(2*d)')(c=299792458, d=real_dist)

In [113]:
L

(0.23138256040284816, 0.01596065604790186)

In [53]:
real_dist

(21594231.03448276, 1301824.6072136145)

In [54]:
dist

(37.6, 2.2199099080818567)

In [56]:
all_peaks[1::2]

array([ 88, 204, 313, 443, 574])

In [178]:
def find_distances(ydata, thresh, num_peaks, new_unit):
    all_peaks, _ = find_peaks(ydata, height=thresh, distance=10)

    num_batches = len(all_peaks) / num_peaks
    peak_batches = np.split(all_peaks, num_batches)

    
    xdata = np.arange(0, len(ydata))
    plt.clf()
    plt.plot(xdata, ydata, label='Spektrum')
    plt.plot(xdata[all_peaks], ydata[all_peaks], "x",
            color='red', label='Peaks')
    plt.xlabel('Zeit [u]')
    plt.ylabel('Amplitude [mV]')
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.show()
    
    dists = np.concatenate([d[1:] - d[:-1] for d in peak_batches])
    dist = dists.mean(), dists.std()/np.sqrt(len(dists))
    return SecondaryValue('u*d')(u=new_unit, d=dist)
mode_dist = SecondaryValue('c/(2*L)', defaults=dict(c=29979245800))    

In [186]:
off_1 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/offener_HeNe/Waveform.txt',
            skiprows=5)
unit_1 = calibrate_unit(off_1, 79)


In [187]:
find_distances(off_1, 50, 3, unit_1)


(187973102.89389068, 12894642.34716403)

In [163]:
mode_dist(L=(80,.5))

(187370286.25, 1171064.2890625)

In [190]:
off_2 = np.loadtxt('../messungen/Boettcher_Matthes/FSR-Messungen/offener_HeNe/res_laenge_60cm.txt',
            skiprows=5)
unit_2 = calibrate_unit(off_2, 80)
find_distances(off_2, 46, 2, unit_2)


(278877209.3023256, 10543335.772201253)

In [191]:
mode_dist(L=(60,.5))


(249827048.33333334, 2081892.0694444445)